In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import random
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import os
from sklearn.preprocessing import StandardScaler
from numpy.random import seed

In [3]:
os.chdir('..')

In [4]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [5]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [6]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [7]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████| 3125/3125 [00:26<00:00, 117.48it/s]


In [8]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████| 782/782 [00:06<00:00, 129.45it/s]


In [9]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [10]:
train=pd.concat(train_dt)

In [11]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████████████████████████████████| 3125/3125 [00:05<00:00, 526.50it/s]


In [12]:
test=pd.concat(test_dt)

In [13]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 868.38it/s]


In [14]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [15]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [16]:
# make dataset
# x를 시계열 데이터로 변경 시켜주는 함수
def make_dataset(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data

In [17]:
series_train = make_dataset(train_sc)
series_test = make_dataset(test_sc)

In [18]:
series_train.shape, series_test.shape

((3125, 600, 18), (782, 600, 18))

In [19]:
# label one-hot 인코딩
labels = train_label['label'].to_numpy()

In [20]:
import tensorflow as tf

2021-10-07 09:08:56.963604: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [21]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [22]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [23]:
# 가중치 적용 loss
# https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf
# 위 논문 참고
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

# 모델을 만들어 주는 함수
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=128, kernel_size=9, padding='same', input_shape=[600, 18]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(filters=256, kernel_size=6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(filters=128, kernel_size=3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [24]:
def make_callbacks(i):
    # checkpoint path
    # 중간중간 모델의 weight를 저장할 경로 설정
    ckpt_name = str(i) + '_fold_cnn_ckpt.hdf5'
    checkpoint_dir_path = os.path.join('checkpoint/best/k_fold_cnn')
    checkpoint_path = os.path.join(checkpoint_dir_path, ckpt_name)
    
    # callback 함수 목록
    callbacks_list = [
        # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
        tf.keras.callbacks.ModelCheckpoint(
            filepath = checkpoint_path,
            monitor='val_loss',
            mode='min',
            save_weights_only=True,
            save_best_only=True
        ),
        # 8번 동안 val_loss의 향상이 없으면 훈련 종료
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            verbose=1, 
            patience=8
        ),
        tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
    ]
    
    return callbacks_list, checkpoint_path

In [25]:
from sklearn.model_selection import StratifiedKFold

In [26]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []
for i, (train, val) in enumerate(split.split(series_train, labels)):
    callbacks_list, checkpoint_path = make_callbacks(i)
    print('Fold', i)
    print('#'*20)

    train_dataset = make_train(series_train[train], labels[train])
    val_dataset = make_val(series_train[val], labels[val])

    model = base()

    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    results.append(result)
    accs.append(acc)
    models.append(model)

    del model, train_dataset, val_dataset

Fold 0
####################


2021-10-07 09:08:57.574165: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-10-07 09:08:57.603006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-07 09:08:57.603509: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2021-10-07 09:08:57.603527: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-10-07 09:08:57.604678: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-10-07 09:08:57.605470: I tensorflow/stream_executor/platform/default/

Epoch 1/100


2021-10-07 09:08:58.998019: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-10-07 09:08:59.117581: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


44/44 [==============================] - 1s 34ms/step - loss: 2.8274 - accuracy: 0.4584 - val_loss: 3.3931 - val_accuracy: 0.2717
Epoch 2/100
44/44 [==============================] - 1s 28ms/step - loss: 1.9541 - accuracy: 0.5452 - val_loss: 1.8818 - val_accuracy: 0.5409
Epoch 3/100
44/44 [==============================] - 1s 27ms/step - loss: 1.7250 - accuracy: 0.5786 - val_loss: 1.9380 - val_accuracy: 0.5505
Epoch 4/100
44/44 [==============================] - 1s 29ms/step - loss: 1.5330 - accuracy: 0.6177 - val_loss: 1.7688 - val_accuracy: 0.5647
Epoch 5/100
44/44 [==============================] - 1s 28ms/step - loss: 1.3786 - accuracy: 0.6422 - val_loss: 1.5683 - val_accuracy: 0.5985
Epoch 6/100
44/44 [==============================] - 1s 29ms/step - loss: 1.2450 - accuracy: 0.6725 - val_loss: 1.3714 - val_accuracy: 0.6309
Epoch 7/100
44/44 [==============================] - 1s 28ms/step - loss: 1.1251 - accuracy: 0.7016 - val_loss: 1.2434 - val_accuracy: 0.6600
Epoch 8/100
44/44 

...

44/44 [==============================] - 1s 28ms/step - loss: 0.2860 - accuracy: 0.9204 - val_loss: 0.3418 - val_accuracy: 0.9033
Epoch 34/100
44/44 [==============================] - 1s 27ms/step - loss: 0.2775 - accuracy: 0.9243 - val_loss: 0.3446 - val_accuracy: 0.9054
Epoch 35/100
44/44 [==============================] - 1s 28ms/step - loss: 0.2614 - accuracy: 0.9307 - val_loss: 0.3253 - val_accuracy: 0.9051
Epoch 36/100
44/44 [==============================] - 1s 28ms/step - loss: 0.2474 - accuracy: 0.9342 - val_loss: 0.3133 - val_accuracy: 0.9097
Epoch 37/100
44/44 [==============================] - 1s 27ms/step - loss: 0.2390 - accuracy: 0.9392 - val_loss: 0.3145 - val_accuracy: 0.9104
Epoch 38/100
44/44 [==============================] - 1s 27ms/step - loss: 0.2324 - accuracy: 0.9424 - val_loss: 0.2752 - val_accuracy: 0.9229
Epoch 39/100
44/44 [==============================] - 1s 27ms/step - loss: 0.2228 - accuracy: 0.9449 - val_loss: 0.3026 - val_accuracy: 0.9214
Epoch 40/100

Epoch 00085: early stopping
44/44 [==============================] - 0s 7ms/step - loss: 0.1116 - accuracy: 0.9787
Fold 8
####################
Epoch 1/100
44/44 [==============================] - 1s 32ms/step - loss: 2.8177 - accuracy: 0.4653 - val_loss: 3.3023 - val_accuracy: 0.2769
Epoch 2/100
44/44 [==============================] - 1s 29ms/step - loss: 1.9490 - accuracy: 0.5492 - val_loss: 1.8668 - val_accuracy: 0.5407
Epoch 3/100
44/44 [==============================] - 1s 25ms/step - loss: 1.7167 - accuracy: 0.5894 - val_loss: 1.8906 - val_accuracy: 0.5485
Epoch 4/100
44/44 [==============================] - 1s 29ms/step - loss: 1.5280 - accuracy: 0.6189 - val_loss: 1.7411 - val_accuracy: 0.5667
Epoch 5/100
44/44 [==============================] - 1s 29ms/step - loss: 1.3785 - accuracy: 0.6481 - val_loss: 1.5152 - val_accuracy: 0.6018
Epoch 6/100
44/44 [==============================] - 1s 27ms/step - loss: 1.2499 - accuracy: 0.6687 - val_loss: 1.3643 - val_accuracy: 0.6289
Epoch

Epoch 55/100
44/44 [==============================] - 1s 28ms/step - loss: 0.1097 - accuracy: 0.9776 - val_loss: 0.1676 - val_accuracy: 0.9641
Epoch 56/100
44/44 [==============================] - 1s 28ms/step - loss: 0.1045 - accuracy: 0.9797 - val_loss: 0.1660 - val_accuracy: 0.9652
Epoch 57/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0992 - accuracy: 0.9854 - val_loss: 0.1635 - val_accuracy: 0.9655
Epoch 58/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0971 - accuracy: 0.9851 - val_loss: 0.1603 - val_accuracy: 0.9637
Epoch 59/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0966 - accuracy: 0.9844 - val_loss: 0.1600 - val_accuracy: 0.9634
Epoch 60/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0932 - accuracy: 0.9851 - val_loss: 0.1598 - val_accuracy: 0.9637
Epoch 61/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0898 - accuracy: 0.9868 - val_loss: 0.1704 - val_accuracy: 0.9652

44/44 [==============================] - 1s 28ms/step - loss: 1.0337 - accuracy: 0.7220 - val_loss: 1.0857 - val_accuracy: 0.7007
Epoch 9/100
44/44 [==============================] - 1s 28ms/step - loss: 0.9489 - accuracy: 0.7433 - val_loss: 0.9853 - val_accuracy: 0.7273
Epoch 10/100
44/44 [==============================] - 1s 28ms/step - loss: 0.8768 - accuracy: 0.7586 - val_loss: 0.9238 - val_accuracy: 0.7458
Epoch 11/100
44/44 [==============================] - 1s 28ms/step - loss: 0.8099 - accuracy: 0.7807 - val_loss: 0.8498 - val_accuracy: 0.7618
Epoch 12/100
44/44 [==============================] - 1s 28ms/step - loss: 0.7562 - accuracy: 0.7874 - val_loss: 0.7862 - val_accuracy: 0.7789
Epoch 13/100
44/44 [==============================] - 1s 27ms/step - loss: 0.7092 - accuracy: 0.7945 - val_loss: 0.7311 - val_accuracy: 0.8041
Epoch 14/100
44/44 [==============================] - 1s 29ms/step - loss: 0.6698 - accuracy: 0.8098 - val_loss: 0.6909 - val_accuracy: 0.8105
Epoch 15/100


44/44 [==============================] - 1s 26ms/step - loss: 0.0738 - accuracy: 0.9943 - val_loss: 0.1950 - val_accuracy: 0.9403
Epoch 65/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0731 - accuracy: 0.9911 - val_loss: 0.1607 - val_accuracy: 0.9559
Epoch 66/100
43/44 [============================>.] - ETA: 0s - loss: 0.0691 - accuracy: 0.9927
Epoch 00066: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
44/44 [==============================] - 1s 27ms/step - loss: 0.0695 - accuracy: 0.9929 - val_loss: 0.1597 - val_accuracy: 0.9581
Epoch 67/100
44/44 [==============================] - 1s 30ms/step - loss: 0.0658 - accuracy: 0.9922 - val_loss: 0.1525 - val_accuracy: 0.9602
Epoch 68/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0643 - accuracy: 0.9936 - val_loss: 0.1481 - val_accuracy: 0.9637
Epoch 69/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0600 - accuracy: 0.9932 - val_loss: 0.1474 - val_accuracy:

In [27]:
results

[0.10935653001070023,
 0.09767921268939972,
 0.12388578057289124,
 0.1418388932943344,
 0.11780686676502228,
 0.11251591145992279,
 0.1136917695403099,
 0.1116301417350769,
 0.11207565665245056,
 0.11264929920434952]

In [28]:
accs

[0.970483660697937,
 0.9743954539299011,
 0.9644381403923035,
 0.9665718078613281,
 0.9701279997825623,
 0.9697831273078918,
 0.9726271033287048,
 0.9786704778671265,
 0.9722716212272644,
 0.9747600555419922]

In [29]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [30]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join('sub/test(1007).csv')     # 저장 경로
sub.to_csv(save_path, index=False)